In [13]:
from numpy import save, load
from pandas import read_csv
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import Conv1D, SpatialDropout1D
from keras.layers import Convolution1D, Dense,Activation
from keras.models import Input, Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,GlobalAveragePooling1D,TimeDistributed, MaxPooling1D
import tensorflow as tf
import numpy as np
import keras.backend as K
import keras.layers
from keras import optimizers
from keras.layers import Activation, Lambda
from keras.layers import Convolution1D, Dense
from keras.models import Input, Model
from typing import List, Tuple

In [2]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
x_train = np.load('gdrive/My Drive/dataset5/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset5/trainy.npy')
x_test = np.load('gdrive/My Drive/dataset5/testX.npy')
y_test = np.load('gdrive/My Drive/dataset5/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Mounted at /content/drive
Mounted at /content/gdrive
(66542, 256, 6) (66542, 2) (7600, 256, 6) (7600, 2)


In [4]:
# reshape data into time steps of sub-sequences
n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
n_steps, n_length = 4, 64
X_train = x_train.reshape((x_train.shape[0], n_steps, n_length, n_features))
X_test = x_test.reshape((x_test.shape[0], n_steps, n_length, n_features))

In [5]:

from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)


In [14]:
model2 = Sequential()
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
model2.add(TimeDistributed(Dropout(0.5)))
model2.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model2.add(TimeDistributed(Flatten()))
model2.add(LSTM(100))
model2.add(Dropout(0.5))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(n_outputs, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()
callbacks = [keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_15 (TimeDi  (None, None, 62, 64)     1216      
 stributed)                                                      
                                                                 
 time_distributed_16 (TimeDi  (None, None, 60, 64)     12352     
 stributed)                                                      
                                                                 
 time_distributed_17 (TimeDi  (None, None, 60, 64)     0         
 stributed)                                                      
                                                                 
 time_distributed_18 (TimeDi  (None, None, 30, 64)     0         
 stributed)                                                      
                                                                 
 time_distributed_19 (TimeDi  (None, None, 1920)      

In [8]:
# fit network
history = model2.fit(X_training, y_training, epochs=200, verbose=True, validation_data=(X_validation, y_validation), batch_size=100)

Epoch 1/200
533/533 [==============================] - 10s 19ms/step - loss: 0.4419 - accuracy: 0.7922 - val_loss: 0.3517 - val_accuracy: 0.8533
Epoch 2/200
533/533 [==============================] - 9s 17ms/step - loss: 0.3102 - accuracy: 0.8774 - val_loss: 0.2516 - val_accuracy: 0.9050
Epoch 3/200
533/533 [==============================] - 9s 18ms/step - loss: 0.2312 - accuracy: 0.9125 - val_loss: 0.2028 - val_accuracy: 0.9237
Epoch 4/200
533/533 [==============================] - 10s 18ms/step - loss: 0.1792 - accuracy: 0.9338 - val_loss: 0.1679 - val_accuracy: 0.9386
Epoch 5/200
533/533 [==============================] - 9s 17ms/step - loss: 0.1456 - accuracy: 0.9463 - val_loss: 0.1485 - val_accuracy: 0.9464
Epoch 6/200
533/533 [==============================] - 9s 18ms/step - loss: 0.1208 - accuracy: 0.9571 - val_loss: 0.1398 - val_accuracy: 0.9530
Epoch 7/200
533/533 [==============================] - 9s 18ms/step - loss: 0.1015 - accuracy: 0.9639 - val_loss: 0.1243 - val_accurac

In [9]:
# evaluate model
loss, accuracy = model2.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model2.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9784
Testing Accuracy: 0.8707
